# 第8章 循环神经网络

## 8.1 序列模型

### 练习 8.1.1

改进本节实验中的模型。
1. 是否包含了过去$4$个以上的观测结果？真实值需要是多少个？
1. 如果没有噪音，需要多少个过去的观测结果？提示：把$\sin$和$\cos$写成微分方程。
1. 可以在保持特征总数不变的情况下合并旧的观察结果吗？这能提高正确度吗？为什么？
1. 改变神经网络架构并评估其性能。

**解答：** 

### 练习 8.1.2

一位投资者想要找到一种好的证券来购买。他查看过去的回报，以决定哪一种可能是表现良好的。这一策略可能会出什么问题呢？

**解答:** 

### 练习 8.1.3

时间是向前推进的因果模型在多大程度上适用于文本呢？

**解答：** 

### 练习 8.1.4

举例说明什么时候可能需要隐变量自回归模型来捕捉数据的动力学模型。

**解答：** 

## 8.2 文本预处理 

### 练习 8.2.1

词元化是一个关键的预处理步骤，它因语言而异。尝试找到另外三种常用的词元化文本的方法。

**解答：** 

### 练习 8.2.2 

在本节的实验中，将文本词元为单词和更改`Vocab`实例的`min_freq`参数。这对词表大小有何影响？

**解答：** 

## 8.3 语言模型和数据集

### 练习 8.3.1

假设训练数据集中有$100,000$个单词。一个四元语法需要存储多少个词频和相邻多词频率？

**解答：** 

### 练习 8.3.2

我们如何对一系列对话建模？

**解答:** 

### 练习 8.3.3

一元语法、二元语法和三元语法的齐普夫定律的指数是不一样的，能设法估计么？

**解答：** 

### 练习 8.3.4

想一想读取长序列数据的其他方法？

**解答：** 

### 练习 8.3.5

考虑一下我们用于读取长序列的随机偏移量。
1. 为什么随机偏移量是个好主意？
1. 它真的会在文档的序列上实现完美的均匀分布吗？
1. 要怎么做才能使分布更均匀？

**解答：** 

### 练习 8.3.6

如果我们希望一个序列样本是一个完整的句子，那么这在小批量抽样中会带来怎样的问题？如何解决？

**解答：** 

## 8.4 循环神经网络 

### 练习 8.4.1

如果我们使用循环神经网络来预测文本序列中的下一个字符，那么任意输出所需的维度是多少？

**解答：** 

### 练习 8.4.2

为什么循环神经网络可以基于文本序列中所有先前的词元，在某个时间步表示当前词元的条件概率？

**解答：** 

### 练习 8.4.3

如果基于一个长序列进行反向传播，梯度会发生什么状况？

**解答：** 

### 练习 8.4.4

与本节中描述的语言模型相关的问题有哪些？

**解答：** 

## 8.5 循环神经网络的从零开始实现 

### 练习 8.5.1

尝试说明独热编码等价于为每个对象选择不同的嵌入表示。

**解答：**

### 练习 8.5.2

通过调整超参数（如迭代周期数、隐藏单元数、小批量数据的时间步数、学习率等）来改善困惑度。
1. 困惑度可以降到多少？
1. 用可学习的嵌入表示替换独热编码，是否会带来更好的表现？
1. 如果用H.G.Wells的其他书作为数据集时效果如何，例如[*世界大战*](http://www.gutenberg.org/ebooks/36)？

**解答：** 

### 练习 8.5.3

修改预测函数，例如使用采样，而不是选择最有可能的下一个字符。
1. 会发生什么？
1. 调整模型使之偏向更可能的输出，例如，当$\alpha > 1$，从$q(x_t \mid x_{t-1}, \ldots, x_1) \propto P(x_t \mid x_{t-1}, \ldots, x_1)^\alpha$中采样。

**解答：** 

### 练习 8.5.4

在不裁剪梯度的情况下运行本节中的代码会发生什么？

**解答：** 

### 练习 8.5.5

更改顺序划分，使其不会从计算图中分离隐状态。运行时间会有变化吗？困惑度呢？

**解答：** 

### 练习 8.5.6

用ReLU替换本节中使用的激活函数，并重复本节中的实验。我们还需要梯度裁剪吗？为什么？

**解答：** 

## 8.6 循环神经网络的简洁实现 

### 练习 8.6.1

尝试使用高级API，能使循环神经网络模型过拟合吗？

**解答：** 

### 练习 8.6.2

如果在循环神经网络模型中增加隐藏层的数量会发生什么？能使模型正常工作吗？

**解答：** 

### 练习 8.6.3

尝试使用循环神经网络实现8.1节中的自回归模型。

**解答：** 

## 8.7 通过时间反向传播 

### 练习 8.7.1

假设我们拥有一个对称矩阵$\mathbf{M} \in \mathbb{R}^{n \times n}$，其特征值为$\lambda_i$，对应的特征向量是$\mathbf{v}_i$（$i = 1, \ldots, n$）。通常情况下，假设特征值的序列顺序为$|\lambda_i| \geq |\lambda_{i+1}|$。
   1. 证明$\mathbf{M}^k$拥有特征值$\lambda_i^k$。
   1. 证明对于一个随机向量$\mathbf{x} \in \mathbb{R}^n$，$\mathbf{M}^k \mathbf{x}$将有较高概率与$\mathbf{M}$的特征向量$\mathbf{v}_1$在一条直线上。形式化这个证明过程。
   1. 上述结果对于循环神经网络中的梯度意味着什么？

**解答：** 

### 练习 8.7.2

除了梯度截断，还有其他方法来应对循环神经网络中的梯度爆炸吗？

**解答：** 